### Cine21 영화배우 정보 크롤링
* site 주소 : http://www.cine21.com/rank/person
* 개발자 도구에서 XHR 인 것만 필터링 network -> content 페이지의 요청방식 확인
* Request URL : http://www.cine21.com/rank/person/content
* Request Method : Post
* Form Data (1개월치 데이터)
  - section: actor
  - period_start: 2020-07
  - gender: all
  - page: 1
* 1 ~ 2 년치 데이터를 한꺼번에 크롤링 해서 actor.json 파일로 저장하기

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pymongo
from urllib.parse import urljoin

### 정규표현식
- \w 는 문자와 숫자
- * : 0 ~ n (횟수), + : 1 ~ n, ? : 0(zero) or 1
<pre>
test_data ='유아인(2편)'
re.sub('\(\w*\)','',test_data) #결과 : '유아인'
</pre>
- . (period) : 줄바꿈 문자(\n)을 제외한 모든 글자 1개를 의미한다.
- .* 는 문자가 0번 또는 그 이상 반복되는 패턴
- Greedy( .* ) vs Non-Greedy( .*? )
 - Greedy( .* ) (욕심많은) : 모두 선택됨
 - Non-Greedy( .*? ) (욕심없는) : 첫번째 매칭되는 태그만
- [\n\r\t] : \n,\r,\t중  하나를 찾는다.
- re.compile(r'[\n\r\t]')는 해당 정규표현식을 찾아주는 컴파일러를 생성해주는 함수이다.
- compile 함수내의 정규표현식 앞에 항상 r을 붙여주어야 한다.

#### 1단계

In [28]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict = dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2020-07'
formdata_dict['gender'] = 'all'
formdata_dict['page'] = 1
formdata_dict

res = requests.post(actor_url, data=formdata_dict)
#print(res.ok)

actor_item_list = list()
soup = BeautifulSoup(res.text,'html.parser')
for actor in soup.select('li.people_li div.name'):
    #print(actor.text)
    re.sub('\(\w*\)','',actor.text)

for a_tag in soup.select('li.people_li div.name a'):
    actor_link = urljoin(actor_url,a_tag['href'])
    
    res = requests.get(actor_link)
    soup = BeautifulSoup(res.text,'html.parser')
    
    actor_item_dict = dict()
    for li_tag in soup.select('ul.default_info li'):
        # dict의 key에 해당하는 값을 추출한다. dict['직업]'
        actor_item_field = li_tag.select_one('span.tit').text #직업
        
        #dict의 value에 해당하는 값을 추출. dict['직업'] == '배우'
        # <li><span class="tit">직업</span>배우</li>
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) #<li>배우</li>
        actor_item_value = re.sub('<.*?>','',actor_item_value) #베우
        
        '''적용전
        \n https://gangdongwon.com
        '''
        regex = re.compile(r'[\n\r\t]')
        actor_item_value = regex.sub('',actor_item_value)
        '''적용후
        https://gangdongwon.com
        '''
        actor_item_dict[actor_item_field] = actor_item_value
    actor_item_list.append(actor_item_dict)
print(actor_item_list,len(actor_item_list))

[{'직업': '배우', '생년월일': '1981-01-18', '성별': '남', '홈페이지': 'http://www.gangdongwon.com', '신장/체중': '186cm, 68kg', '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)', '취미': '음악감상, 웹서핑, 운동, 신발 모으기', '특기': '축구'}, {'직업': '배우', '생년월일': '1980-02-07', '성별': '여', '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh', '신장/체중': '158cm, 38kg', '학교': '중앙대학교 연극영화', '취미': '바비 인형 수집, 피아노 연주', '특기': '별관찰, 시나리오 쓰기', '소속사': '에이바 앤 엔터테인먼트'}, {'직업': '배우', '생년월일': '2006-03-12', '성별': '여', '홈페이지': 'https://www.facebook.com/leere.vv/'}, {'직업': '배우', '생년월일': '1965-11-06', '성별': '남', '신장/체중': '170cm, 62kg', '학교': '한양대학교 연극영화학 졸업'}, {'직업': '배우', '생년월일': '1982-12-14', '성별': '남'}, {'직업': '배우', '생년월일': '1979-03-20', '성별': '남', '홈페이지': 'https://www.facebook.com/cjesminjae', '학교': '한국예술종합학교'}, {'다른 이름': '엄홍식', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '

#### 2단계
* 1page의 7명의 배우 정보를 출력

In [49]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict = dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2020-07'
formdata_dict['gender'] = 'all'
formdata_dict['page'] = 1
formdata_dict

res = requests.post(actor_url, data=formdata_dict)

soup = BeautifulSoup(res.text,'html.parser')

actor_item_list = list()

actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')
rankings = soup.select('li.people_li span.grade ')

# actor변수가 <div class=name>를 의미함
for index, actor in enumerate(actors):
    actor_item_dict = dict()
    #print(actor)
    # 유아인(2편) 에서 (2편)을 제거하고 저장한다.
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_item_dict['배우이름'] = actor_name
    
    # 흥행지수 값에서 ,(콤마)를 제거하고 정수타입으로 변환해서 저장한다.
    actor_hit = int(hits[index].text.replace(',',''))
    actor_item_dict['흥행지수'] = actor_hit
    
    #출연작
    movie_titles = movies[index].select('li a span')    
    #출연작 여러개의 title을 저장하는 리스트
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    actor_item_dict['출연영화'] = movie_title_list
    
    #순위
    actor_ranking = rankings[index].text
    actor_item_dict['랭킹'] = int(actor_ranking)
    
    #배우의 상세정보를 보기 위해 http get다시 요청
    '''
        <div class=name>
            <a href="/db/person/info/?person_id=18040"> --> actor_detail_url
    '''
    actor_detail_url = actor.select_one('a').attrs['href']
    actor_etail_full_url = urljoin(actor_url,actor_detail_url)
    #print(actor_etail_full_url)
    
    res = requests.get(actor_link)
    soup = BeautifulSoup(res.text,'html.parser')
       
    for li_tag in soup.select('ul.default_info li'):
        # dict의 key에 해당하는 값을 추출한다. dict['직업]'
        actor_item_field = li_tag.select_one('span.tit').text #직업
        
        #dict의 value에 해당하는 값을 추출. dict['직업'] == '배우'
        # <li><span class="tit">직업</span>배우</li>
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) #<li>배우</li>
        actor_item_value = re.sub('<.*?>','',actor_item_value) #베우
        
        '''적용전
        \n https://gangdongwon.com
        '''
        regex = re.compile(r'[\n\r\t]')
        actor_item_value = regex.sub('',actor_item_value)
        '''적용후
        https://gangdongwon.com
        '''
        actor_item_dict[actor_item_field] = actor_item_value
    actor_item_list.append(actor_item_dict)
print(actor_item_list,len(actor_item_list))

[{'배우이름': '강동원', '흥행지수': 159665, '출연영화': ['반도'], '랭킹': 1, '다른 이름': '엄홍식', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '스노우보드, 수영'}, {'배우이름': '이정현', '흥행지수': 139707, '출연영화': ['반도'], '랭킹': 2, '다른 이름': '엄홍식', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '스노우보드, 수영'}, {'배우이름': '이레', '흥행지수': 119748, '출연영화': ['반도'], '랭킹': 3, '다른 이름': '엄홍식', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '스노우보드, 수영'}, {'배우이름': '권해효', '흥행지수': 99795, '출연영화': ['반도', '사이비', '한낮의 피크닉'], '랭킹': 4, '다른 이름': '엄홍식', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '

#### 3단계
* 1,2,3 페이지의 21명 배우의 정보를 출력

In [56]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict = dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2020-07'
formdata_dict['gender'] = 'all'

actor_item_list = list()
for page in range(1,4):
    formdata_dict['page'] = page
    print('----> {}페이지'.format(page))

    res = requests.post(actor_url, data=formdata_dict)

    soup = BeautifulSoup(res.text,'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li span.grade ')

    # actor변수가 <div class=name>를 의미함
    for index, actor in enumerate(actors):
        print('==> {} 번째 배우 출력'.format(index))
        actor_item_dict = dict()
        #print(actor)
        # 유아인(2편) 에서 (2편)을 제거하고 저장한다.
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_item_dict['배우이름'] = actor_name

        # 흥행지수 값에서 ,(콤마)를 제거하고 정수타입으로 변환해서 저장한다.
        actor_hit = int(hits[index].text.replace(',',''))
        actor_item_dict['흥행지수'] = actor_hit

        #출연작
        movie_titles = movies[index].select('li a span')    
        #출연작 여러개의 title을 저장하는 리스트
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_item_dict['출연영화'] = movie_title_list

        #순위
        actor_ranking = rankings[index].text
        actor_item_dict['랭킹'] = int(actor_ranking)

        #배우의 상세정보를 보기 위해 http get다시 요청
        '''
            <div class=name>
                <a href="/db/person/info/?person_id=18040"> --> actor_detail_url
        '''
        actor_detail_url = actor.select_one('a').attrs['href']
        actor_etail_full_url = urljoin(actor_url,actor_detail_url)
        #print(actor_etail_full_url)

        res = requests.get(actor_link)
        soup = BeautifulSoup(res.text,'html.parser')

        for li_tag in soup.select('ul.default_info li'):
            # dict의 key에 해당하는 값을 추출한다. dict['직업]'
            actor_item_field = li_tag.select_one('span.tit').text #직업

            #dict의 value에 해당하는 값을 추출. dict['직업'] == '배우'
            # <li><span class="tit">직업</span>배우</li>
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) #<li>배우</li>
            actor_item_value = re.sub('<.*?>','',actor_item_value) #베우

            '''적용전
            \n https://gangdongwon.com
            '''
            regex = re.compile(r'[\n\r\t]')
            actor_item_value = regex.sub('',actor_item_value)
            '''적용후
            https://gangdongwon.com
            '''
            actor_item_dict[actor_item_field] = actor_item_value
        actor_item_list.append(actor_item_dict)
    #print(actor_item_list,len(actor_item_list))
print(actor_item_list,len(actor_item_list))

----> 1페이지
==> 0 번째 배우 출력
==> 1 번째 배우 출력
==> 2 번째 배우 출력
==> 3 번째 배우 출력
==> 4 번째 배우 출력
==> 5 번째 배우 출력
==> 6 번째 배우 출력
----> 2페이지
==> 0 번째 배우 출력
==> 1 번째 배우 출력
==> 2 번째 배우 출력
==> 3 번째 배우 출력
==> 4 번째 배우 출력
==> 5 번째 배우 출력
==> 6 번째 배우 출력
----> 3페이지
==> 0 번째 배우 출력
==> 1 번째 배우 출력
==> 2 번째 배우 출력
==> 3 번째 배우 출력
==> 4 번째 배우 출력
==> 5 번째 배우 출력
==> 6 번째 배우 출력
[{'배우이름': '강동원', '흥행지수': 159665, '출연영화': ['반도'], '랭킹': 1, '다른 이름': '엄홍식', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '스노우보드, 수영'}, {'배우이름': '이정현', '흥행지수': 139707, '출연영화': ['반도'], '랭킹': 2, '다른 이름': '엄홍식', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '스노우보드, 수영'}, {'배우이름': '이레', '흥행지수': 119748, '출연영화': ['반도'], '랭킹': 3, '다른 이름': '엄홍식', '직업

#### 4단계
* 1년치 데이터 1905명 배우의 정보를 출력

In [58]:
import requests
from bs4 import BeautifulSoup
import re
import pymongo
from urllib.parse import urljoin
from itertools import count

actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict = dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2019-07'
formdata_dict['gender'] = 'all'

actor_item_list = list()
for page in count(1):
    formdata_dict['page'] = page
    print('----> {}페이지'.format(page))

    res = requests.post(actor_url, data=formdata_dict)

    soup = BeautifulSoup(res.text,'html.parser')

    actors = soup.select('li.people_li div.name')
    if len(actors) == 0:
        break
        
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li span.grade ')

    # actor변수가 <div class=name>를 의미함
    for index, actor in enumerate(actors):
        actor_item_dict = dict()
        #print(actor)
        # 유아인(2편) 에서 (2편)을 제거하고 저장한다.
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_item_dict['배우이름'] = actor_name
        print('==> {}({}) 배우 출력'.format(actor_name,len(actor_name)))

        # 흥행지수 값에서 ,(콤마)를 제거하고 정수타입으로 변환해서 저장한다.
        actor_hit = int(hits[index].text.replace(',',''))
        actor_item_dict['흥행지수'] = actor_hit

        #출연작
        movie_titles = movies[index].select('li a span')    
        #출연작 여러개의 title을 저장하는 리스트
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_item_dict['출연영화'] = movie_title_list

        #순위
        actor_ranking = rankings[index].text
        actor_item_dict['랭킹'] = int(actor_ranking)

        #배우의 상세정보를 보기 위해 http get다시 요청
        '''
            <div class=name>
                <a href="/db/person/info/?person_id=18040"> --> actor_detail_url
        '''
        actor_detail_url = actor.select_one('a').attrs['href']
        actor_etail_full_url = urljoin(actor_url,actor_detail_url)
        #print(actor_etail_full_url)

        res = requests.get(actor_link)
        soup = BeautifulSoup(res.text,'html.parser')

        for li_tag in soup.select('ul.default_info li'):
            # dict의 key에 해당하는 값을 추출한다. dict['직업]'
            actor_item_field = li_tag.select_one('span.tit').text #직업

            #dict의 value에 해당하는 값을 추출. dict['직업'] == '배우'
            # <li><span class="tit">직업</span>배우</li>
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) #<li>배우</li>
            actor_item_value = re.sub('<.*?>','',actor_item_value) #베우

            '''적용전
            \n https://gangdongwon.com
            '''
            regex = re.compile(r'[\n\r\t]')
            actor_item_value = regex.sub('',actor_item_value)
            '''적용후
            https://gangdongwon.com
            '''
            actor_item_dict[actor_item_field] = actor_item_value
        actor_item_list.append(actor_item_dict)
    #print(actor_item_list,len(actor_item_list))
print(actor_item_list,len(actor_item_list))

----> 1페이지
==> 이병헌(3) 배우 출력
==> 하정우(3) 배우 출력
==> 마동석(3) 배우 출력
==> 전혜진(3) 배우 출력
==> 이성민(3) 배우 출력
==> 김아중(3) 배우 출력
==> 박정민(3) 배우 출력
----> 2페이지
==> 강기영(3) 배우 출력
==> 조진웅(3) 배우 출력
==> 조정석(3) 배우 출력
==> 배수지(3) 배우 출력
==> 유아인(3) 배우 출력
==> 임윤아(3) 배우 출력
==> 고두심(3) 배우 출력
----> 3페이지
==> 장기용(3) 배우 출력
==> 권상우(3) 배우 출력
==> 곽도원(3) 배우 출력
==> 유해진(3) 배우 출력
==> 최유화(3) 배우 출력
==> 박인환(3) 배우 출력
==> 정해인(3) 배우 출력
----> 4페이지
==> 정유미(3) 배우 출력
==> 김지영(3) 배우 출력
==> 류준열(3) 배우 출력
==> 이봉련(3) 배우 출력
==> 조우진(3) 배우 출력
==> 공유(2) 배우 출력
==> 강동원(3) 배우 출력
----> 5페이지
==> 김종구(3) 배우 출력
==> 박지환(3) 배우 출력
==> 이정현(3) 배우 출력
==> 김상중(3) 배우 출력
==> 이희준(3) 배우 출력
==> 김미경(3) 배우 출력
==> 허성태(3) 배우 출력
----> 6페이지
==> 김병순(3) 배우 출력
==> 박신혜(3) 배우 출력
==> 염정아(3) 배우 출력
==> 배성우(3) 배우 출력
==> 이경영(3) 배우 출력
==> 허준호(3) 배우 출력
==> 권해효(3) 배우 출력
----> 7페이지
==> 황효은(3) 배우 출력
==> 이레(2) 배우 출력
==> 이하늬(3) 배우 출력
==> 김래원(3) 배우 출력
==> 공민정(3) 배우 출력
==> 성유빈(3) 배우 출력
==> 우도환(3) 배우 출력
----> 8페이지
==> 공효진(3) 배우 출력
==> 최성은(3) 배우 출력
==> 박해준(3) 배우 출력
==> 윤경호(3) 배우 출력
==> 조한철(3) 배우

==> 류아벨(3) 배우 출력
----> 64페이지
==> 설혜인(3) 배우 출력
==> 김다올(3) 배우 출력
==> 홍기준(3) 배우 출력
==> 김학선(3) 배우 출력
==> 송영규(3) 배우 출력
==> 이하율(3) 배우 출력
==> 노강민(3) 배우 출력
----> 65페이지
==> 이다슬(3) 배우 출력
==> 박용우(3) 배우 출력
==> 최예나(3) 배우 출력
==> 구성환(3) 배우 출력
==> 한만중(3) 배우 출력
==> 문동환(3) 배우 출력
==> 홍수아(3) 배우 출력
----> 66페이지
==> 김서연(3) 배우 출력
==> 정두홍(3) 배우 출력
==> 한철우(3) 배우 출력
==> 임성철(3) 배우 출력
==> 김병춘(3) 배우 출력
==> 치타(2) 배우 출력
==> 이채연(3) 배우 출력
----> 67페이지
==> 오지혜(3) 배우 출력
==> 이열음(3) 배우 출력
==> 한송희(3) 배우 출력
==> 최재환(3) 배우 출력
==> 안영미(3) 배우 출력
==> 김보윤(3) 배우 출력
==> 김채원(3) 배우 출력
----> 68페이지
==> 김소은(3) 배우 출력
==> 곽희성(3) 배우 출력
==> 지창욱(3) 배우 출력
==> 김홍표(3) 배우 출력
==> 최다니엘(4) 배우 출력
==> 곽진석(3) 배우 출력
==> 류덕환(3) 배우 출력
----> 69페이지
==> 김창옥(3) 배우 출력
==> 고구인(3) 배우 출력
==> 김경희(3) 배우 출력
==> 오지호(3) 배우 출력
==> 최정현(3) 배우 출력
==> 조은지(3) 배우 출력
==> 정은경(3) 배우 출력
----> 70페이지
==> 민형식(3) 배우 출력
==> 최원형(3) 배우 출력
==> 박지현(3) 배우 출력
==> 최정윤(3) 배우 출력
==> 곽동연(3) 배우 출력
==> 윤승아(3) 배우 출력
==> 김소현(3) 배우 출력
----> 71페이지
==> 문성근(3) 배우 출력
==> 안효민(3) 배우 출력
==> 김대중(3) 배우 출력
==>

==> 허준석(3) 배우 출력
----> 125페이지
==> 고소영(3) 배우 출력
==> 이호산(3) 배우 출력
==> 김주영(3) 배우 출력
==> 이소은(3) 배우 출력
==> 박세민(3) 배우 출력
==> 박점례(3) 배우 출력
==> 정진영(3) 배우 출력
----> 126페이지
==> 김유연(3) 배우 출력
==> 유하(2) 배우 출력
==> 강애심(3) 배우 출력
==> 박인수(3) 배우 출력
==> 전숙경(3) 배우 출력
==> 이민자(3) 배우 출력
==> 박세준(3) 배우 출력
----> 127페이지
==> 김하림(3) 배우 출력
==> 길해연(3) 배우 출력
==> 최명빈(3) 배우 출력
==> 이엘(2) 배우 출력
==> 정찬(2) 배우 출력
==> 박수영(3) 배우 출력
==> 이강갑(3) 배우 출력
----> 128페이지
==> 황정순(3) 배우 출력
==> 강은우(3) 배우 출력
==> 윤상현(3) 배우 출력
==> 이광자(3) 배우 출력
==> 김욱(2) 배우 출력
==> 박성빈(3) 배우 출력
==> 조민경(3) 배우 출력
----> 129페이지
==> 임창정(3) 배우 출력
==> 박근형(3) 배우 출력
==> 김가영(3) 배우 출력
==> 김도영(3) 배우 출력
==> 김군자(3) 배우 출력
==> 감소현(3) 배우 출력
==> 황선(2) 배우 출력
----> 130페이지
==> 진영(2) 배우 출력
==> 김재록(3) 배우 출력
==> 최승훈(3) 배우 출력
==> 윤봉길(3) 배우 출력
==> 이주원(3) 배우 출력
==> 강선숙(3) 배우 출력
==> 정영기(3) 배우 출력
----> 131페이지
==> 이주원(3) 배우 출력
==> 강선숙(3) 배우 출력
==> 안기임(3) 배우 출력
==> 신문성(3) 배우 출력
==> 이시후(3) 배우 출력
==> 이재원(3) 배우 출력
==> 장미희(3) 배우 출력
----> 132페이지
==> 최병모(3) 배우 출력
==> 김향기(3) 배우 출력
==> 이인옥(3) 배우 출력
=

==> 최성관(3) 배우 출력
==> 홍인(2) 배우 출력
----> 186페이지
==> 황선화(3) 배우 출력
==> 기영도(3) 배우 출력
==> 김충길(3) 배우 출력
==> 김유정(3) 배우 출력
==> 최성관(3) 배우 출력
==> 홍인(2) 배우 출력
==> 안소희(3) 배우 출력
----> 187페이지
==> 안소희(3) 배우 출력
==> 조안(2) 배우 출력
==> 정휘(2) 배우 출력
==> 오민정(3) 배우 출력
==> 방수일(3) 배우 출력
==> 정한비(3) 배우 출력
==> 변요한(3) 배우 출력
----> 188페이지
==> 진지희(3) 배우 출력
==> 김중기(3) 배우 출력
==> 유장현(3) 배우 출력
==> 김기범(3) 배우 출력
==> 신윤주(3) 배우 출력
==> 정은채(3) 배우 출력
==> 김진희(3) 배우 출력
----> 189페이지
==> 채국희(3) 배우 출력
==> 황승언(3) 배우 출력
==> 김신재(3) 배우 출력
==> 손석구(3) 배우 출력
==> 장동직(3) 배우 출력
==> 박지홍(3) 배우 출력
==> 이지희(3) 배우 출력
----> 190페이지
==> 강현중(3) 배우 출력
==> 서갑숙(3) 배우 출력
==> 최진호(3) 배우 출력
==> 김도현(3) 배우 출력
==> 김가희(3) 배우 출력
==> 윤다경(3) 배우 출력
==> 김상순(3) 배우 출력
----> 191페이지
==> 송영인(3) 배우 출력
==> 하연주(3) 배우 출력
==> 박혜수(3) 배우 출력
==> 유지연(3) 배우 출력
==> 안예인(3) 배우 출력
==> 고수(2) 배우 출력
==> 김새론(3) 배우 출력
----> 192페이지
==> 박기태(3) 배우 출력
==> 최원(2) 배우 출력
==> 최준우(3) 배우 출력
==> 민지아(3) 배우 출력
==> 이웅빈(3) 배우 출력
==> 이윤회(3) 배우 출력
==> 이주현(3) 배우 출력
----> 193페이지
==> 최원(2) 배우 출력
==> 최준우(3) 배우 출력
==

==> 김주현(3) 배우 출력
==> 이동욱(3) 배우 출력
==> 이요섭(3) 배우 출력
==> 손지현(3) 배우 출력
==> 마민지(3) 배우 출력
==> 박혜숙(3) 배우 출력
----> 249페이지
==> 김혜정(3) 배우 출력
==> 이민정(3) 배우 출력
==> 서이안(3) 배우 출력
==> 임정은(3) 배우 출력
==> 김윤지(3) 배우 출력
==> 한성수(3) 배우 출력
==> 여호민(3) 배우 출력
----> 250페이지
==> 장유상(3) 배우 출력
==> 김형묵(3) 배우 출력
==> 허정은(3) 배우 출력
==> 심원철(3) 배우 출력
==> 김혜정(3) 배우 출력
==> 이민정(3) 배우 출력
==> 서이안(3) 배우 출력
----> 251페이지
==> 김도희(3) 배우 출력
==> 김세원(3) 배우 출력
==> 백현진(3) 배우 출력
==> 유순철(3) 배우 출력
==> 채연희(3) 배우 출력
==> 남철(2) 배우 출력
==> 오기환(3) 배우 출력
----> 252페이지
==> 지대한(3) 배우 출력
==> 윤대열(3) 배우 출력
==> 정미남(3) 배우 출력
==> 윤정희(3) 배우 출력
==> 정성모(3) 배우 출력
==> 한재덕(3) 배우 출력
==> 김도희(3) 배우 출력
----> 253페이지
==> 정성모(3) 배우 출력
==> 김도희(3) 배우 출력
==> 김세원(3) 배우 출력
==> 백현진(3) 배우 출력
==> 유순철(3) 배우 출력
==> 박현영(3) 배우 출력
==> 이영미(3) 배우 출력
----> 254페이지
==> 김춘기(3) 배우 출력
==> 이미소(3) 배우 출력
==> 서동원(3) 배우 출력
==> 주석희(3) 배우 출력
==> 신우희(3) 배우 출력
==> 임정환(3) 배우 출력
==> 동하(2) 배우 출력
----> 255페이지
==> 이준동(3) 배우 출력
==> 박지나(3) 배우 출력
==> 이호재(3) 배우 출력
==> 정인하(3) 배우 출력
==> 박선희(3) 배우 출력
==> 유준상(3)

In [59]:
actor_final_list = [actor for actor in actor_item_list if len(actor['배우이름']) < 10]
print(len(actor_final_list))

1903


In [15]:
test_data = """홈페이지

http://www.gangdongwon.com

신장/체중"""
regex = re.compile(r'[\n\r\t]')
regex.sub('',test_data)

'홈페이지http://www.gangdongwon.com신장/체중'